In [1]:
# importing the libraries
import pandas as pd 
import time
from datetime import datetime, timedelta  
import plotly.express as px

In [2]:
# declaring the variables 
StartDate = datetime.strptime('2021-10-21', "%Y-%m-%d")
Delay_Cost_Dollars_Per_Day = 500 
FilePath = r'' # specifying the path of the file
# providing the file name and extension
FileName_and_Ext = 'Dataset.xlsx'

In [3]:
# function to load and prepare the data
def Load_and_Prep_Data(FilePath,FileName_and_Ext):
    # pulling the data
    col_names = columns = ['Machines','Products', 'Hourly_Output','Color_Changeover' ,'Product_Changeover','Machine_Startup_Time','Machine_Shutdown_Time','Hourly_Cost']
    Machine_DF = pd.read_excel(FilePath+FileName_and_Ext,engine='openpyxl',usecols=range(len(col_names)),names=col_names)
    Machine_DF['Per_Unit_Cost'] = Machine_DF['Hourly_Cost'] / Machine_DF['Hourly_Output']
    # pulling the job data
    col_names = columns=['Product','Color','Job_Number','Qty_Needed','Due_Dates']
    Jobs_DF = pd.read_excel(FilePath+FileName_and_Ext,sheet_name='Demand',engine='openpyxl',usecols=range(len(col_names)),names=col_names)
    # order By due date
    Jobs_DF['Due_Dates'] = pd.to_datetime(Jobs_DF['Due_Dates'])
    Jobs_DF = Jobs_DF.sort_values(by=['Due_Dates','Product','Color'])
    # creating a new DataFrame for production Scheduling
    Production_Schedule_DF = pd.DataFrame(columns=['Machine','Job','Product','Color','Qty','Start_Date','Duration','End_Date','Production_Late','Days_Late'])
    return Machine_DF, Jobs_DF, Production_Schedule_DF

In [4]:
def Calculated_Fields(row):
    # calculating changeover, and determining the starting date
    if len(str(row['End_Date'])) <= 3 or row['End_Date'] != row['End_Date']:
        Change_or_Startup_Required = 'Machine_Startup'
        Change_or_Startup_Time = row['Machine_Startup_Time']
        Date_Start = StartDate # overriding default
    elif row['Products']  != row['Product']:
        Change_or_Startup_Required = 'Product Changeover'
        Change_or_Startup_Time = row['Product_Changeover']
        Date_Start = row['End_Date']
    elif row ['Color']  != row['Color2Make']:
        Change_or_Startup_Required = 'Color Changeover'
        Change_or_Startup_Time = row['Color_Changeover']
        Date_Start = row['End_Date']
    else:
        Change_or_Startup_Required = 'No Changeover'
        Change_or_Startup_Time = int(0)
        Date_Start = row['End_Date']
    # calculate runtime and duration
    Product_Only_Runtime_Hours = int(row['Qty_Required']/row['Hourly_Output'])
    Total_Duration_Hours = int(Product_Only_Runtime_Hours + Change_or_Startup_Time)
    # calculating end date and determining if job would be done before due date
    Production_End_Date = Date_Start + timedelta(hours=Total_Duration_Hours)
    Production_Late = Production_End_Date > row['Due_Date']
    if Production_Late == True:
        Days_Late = (Production_End_Date - row['Due_Date']).days
        Delay_Total_Cost = Delay_Cost_Dollars_Per_Day * Days_Late
    else:
        Days_Late = 0
        Delay_Total_Cost = 0
    # calculating total cost 
    Total_Run_Cost = (row['Hourly_Cost'] * Total_Duration_Hours) + Delay_Total_Cost     # inculding delay cost
    return Change_or_Startup_Required, Change_or_Startup_Time, Product_Only_Runtime_Hours, Total_Duration_Hours, Total_Run_Cost,Date_Start,Production_End_Date,Production_Late, Days_Late

In [5]:
def Set_Schedule(DF):
    Job_Prod_Plan = [DF.Machines.iloc[0],DF.Job.iloc[0],DF.Products.iloc[0],DF.Color2Make.iloc[0],
                     DF.Qty_Required.iloc[0],DF.Date_Start.iloc[0],DF.Total_Duration_Hours.iloc[0],
                     DF.Production_End_Date.iloc[0],DF.Production_Late.iloc[0],DF.Days_Late.iloc[0]]
    Production_Schedule_DF.loc[len(Production_Schedule_DF)] = Job_Prod_Plan
    
def Run_Scheduler():
    for Job in Jobs_DF.itertuples(index=False):
        # list of all machines
        Machine_Options_DF = (Machine_DF[Machine_DF['Products'] == Job.Product])
        Machine_Options_DF = Machine_Options_DF.sort_values(by='Per_Unit_Cost',ascending=True)
        # pulling all the options
        Machine_Options_DF = (Machine_DF[Machine_DF['Products'] == Job.Product]).copy()
        Machine_Options_DF['Color2Make'] = Job.Color
        Machine_Options_DF['Qty_Required'] = Job.Qty_Needed
        Machine_Options_DF['Due_Date'] = Job.Due_Dates
        Machine_Options_DF['Job'] = Job.Job_Number
        # creating a temporary DataFrame with the recent records
        try:
            Last_Product_DF = Production_Schedule_DF[['Machine','Product','Color','End_Date']].sort_values('End_Date').groupby('Machine').tail(1) 
        except:
            Last_Product_DF = pd.DataFrame(columns=['Machine','Product','Color','End_Date'])
        # pulling the Data
        Merged_DF = Machine_Options_DF.merge(Last_Product_DF,how='left',left_on='Machines',right_on='Machine')
        Merged_DF[['Change_or_Startup_Required', 'Change_or_Startup_Time', 'Product_Only_Runtime_Hours', 'Total_Duration_Hours', 'Total_Run_Cost','Date_Start','Production_End_Date','Production_Late', 'Days_Late']] = Merged_DF.apply(Calculated_Fields, axis=1, result_type="expand")
        # sorting by lowest cost
        if len(Merged_DF) > 1:
            Merged_DF = Merged_DF.sort_values(by='Total_Run_Cost',ascending=True).head(1)
        else:
            Merged_DF = Merged_DF
            
        Set_Schedule(Merged_DF)
        

In [6]:
# calling the function and getting output in variables
Machine_DF, Jobs_DF, Production_Schedule_DF = Load_and_Prep_Data(FilePath,FileName_and_Ext)

In [7]:
# printing the scheduled queries
Run_Scheduler()
Production_Schedule_DF

,Machine,Job,Product,Color,Qty,Start_Date,Duration,End_Date,Production_Late,Days_Late
0,M18,27,A,O/G,15000,2021-10-21 00:00:00,6,2021-10-21 06:00:00,True,11
1,M14,35,D,MG,20000,2021-10-21 00:00:00,8,2021-10-21 08:00:00,True,9
2,M11,2,B,SG,20000,2021-10-21 00:00:00,10,2021-10-21 10:00:00,True,7
3,M12,34,C,PR,15000,2021-10-21 00:00:00,8,2021-10-21 08:00:00,True,7
4,M11,41,B,FG,20000,2021-10-21 10:00:00,10,2021-10-21 20:00:00,True,2
5,M12,30,C,B,15000,2021-10-21 08:00:00,8,2021-10-21 16:00:00,True,2
6,M10,16,C,SG,20000,2021-10-21 00:00:00,8,2021-10-21 08:00:00,True,1
7,M20,48,D,PU,20000,2021-10-21 00:00:00,9,2021-10-21 09:00:00,True,1
8,M14,9,D,MG,15000,2021-10-21 08:00:00,3,2021-10-21 11:00:00,True,0
9,M19,10,E,MR,20000,2021-10-21 00:00:00,8,2021-10-21 08:00:00,False,0


In [ ]:
Production_Schedule_DF['Task'] = 'T#' + Production_Schedule_DF['Job'].astype(str) + ' | ' + Production_Schedule_DF['Product'].astype(str) + ' | ' + Production_Schedule_DF['Color'].astype(str)
fig = px.timeline(Production_Schedule_DF, x_start="Start_Date", x_end="End_Date", y="Task", color="Machine")
fig.update_yaxes(autorange="reversed")
fig.show()

In [ ]:
Production_Schedule_DF['Task'] = 'T#' + Production_Schedule_DF['Job'].astype(str) + ' | ' + Production_Schedule_DF['Product'].astype(str) + ' | ' + Production_Schedule_DF['Color'].astype(str)
fig = px.timeline(Production_Schedule_DF, x_start="Start_Date", x_end="End_Date", y="Machine", color="Production_Late",color_discrete_sequence=px.colors.qualitative.Set1)
fig.update_yaxes(autorange="reversed")
fig.update_yaxes(categoryorder='category ascending')
fig.update_layout(xaxis=dict(rangeslider=dict(visible=True)))
fig.show()

In [ ]:
Production_Schedule_DF['Task'] = 'T#' + Production_Schedule_DF['Job'].astype(str) + ' | ' + Production_Schedule_DF['Product'].astype(str) + ' | ' + Production_Schedule_DF['Color'].astype(str)
fig = px.timeline(Production_Schedule_DF, x_start="Start_Date", x_end="End_Date", y="Machine", color="Task")
fig.update_yaxes(autorange="reversed")
fig.update_yaxes(categoryorder='category ascending')
fig.update_layout(xaxis=dict(rangeslider=dict(visible=True)))
fig.show()